In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only, encoder_with_convs_and_symmetry_new

from tf_lab.nips.helper import average_per_class, zero_mean_half_sphere

from tf_lab.point_clouds.point_net_classifier import PointNetClassifier
from tf_lab.point_clouds.convenience import classify_pclouds

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
model_net = '40'
verbose = True
n_threads = 20
put_in_usphere = True
n_pc_points = 2048

In [5]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_points))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [6]:
pc, model_names, train_classes = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)
labels = np.array(classes_to_integers(int(model_net), train_classes)[1])

if put_in_usphere:
    pc = zero_mean_half_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, test_classes = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)
labels = np.array(classes_to_integers(int(model_net), test_classes)[1])

if put_in_usphere:
    pc = zero_mean_half_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [7]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [8]:
experiment_name = 'sup_clf_1'

experiment_name = '_'.join([experiment_name, 'mnet', model_net,  'pts', str(n_pc_points)])

train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/sup_clf')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir);

In [11]:
n_input = [n_pc_points, 3]
n_classes = int(model_net)

encoder = encoder_with_convs_and_symmetry_new
decoder = decoder_with_fc_only

decoder_args = {'layer_sizes': [256, 512, n_classes], 'b_norm': False}


encoder_args = {'n_filters': [64, 128, 256, 256],
                'filter_sizes': [1, 1, 1, 1],
                'strides': [1, 1, 1, 1],
                'b_norm': True
               }

conf = Conf(        
            n_input = n_input,
            n_output = [n_classes],
            training_epochs = 200,
            batch_size = 50,
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = 20,
            learning_rate = 0.0005,
            z_rotate = True, 
            encoder = encoder,
            decoder = decoder,
            encoder_args = encoder_args,
            decoder_args = decoder_args
            )


conf.one_hot = False
conf.experiment_name = experiment_name
conf.save(osp.join(conf.train_dir, 'configuration'))    
    
reset_tf_graph()
pnet_clf = PointNetClassifier(experiment_name, conf)

In [12]:
c = conf
for _ in xrange(c.training_epochs):
    loss, duration = pnet_clf._single_epoch_train(train_data, c)
    epoch = int(pnet_clf.sess.run(pnet_clf.epoch.assign_add(tf.constant(1.0))))
    print 'TRAIN epoch: %d loss: %f' % (epoch, loss)
    if epoch % 2 == 0:
        avg_ac = classify_pclouds(pnet_clf, test_data.point_clouds, 100, test_data.labels)[1]
        print 'TEST epoch: %d accuracy: %f' % (epoch, avg_ac)

TRAIN epoch: 1 loss: 2.261164
TRAIN epoch: 2 loss: 1.449835
TEST epoch: 2 accuracy: 0.515606
TRAIN epoch: 3 loss: 1.214170
TRAIN epoch: 4 loss: 1.103653
TEST epoch: 4 accuracy: 0.506688
TRAIN epoch: 5 loss: 1.033509
TRAIN epoch: 6 loss: 0.933533
TEST epoch: 6 accuracy: 0.580057
TRAIN epoch: 7 loss: 0.882647
TRAIN epoch: 8 loss: 0.883759
TEST epoch: 8 accuracy: 0.442643
TRAIN epoch: 9 loss: 0.808734
TRAIN epoch: 10 loss: 0.783823
TEST epoch: 10 accuracy: 0.582083
TRAIN epoch: 11 loss: 0.762333
TRAIN epoch: 12 loss: 0.741029
TEST epoch: 12 accuracy: 0.666396
TRAIN epoch: 13 loss: 0.710247
TRAIN epoch: 14 loss: 0.710340
TEST epoch: 14 accuracy: 0.628699
TRAIN epoch: 15 loss: 0.703854
TRAIN epoch: 16 loss: 0.661073
TEST epoch: 16 accuracy: 0.543981
TRAIN epoch: 17 loss: 0.663848
TRAIN epoch: 18 loss: 0.655406
TEST epoch: 18 accuracy: 0.599919
TRAIN epoch: 19 loss: 0.649717
TRAIN epoch: 20 loss: 0.613144
TEST epoch: 20 accuracy: 0.467775
TRAIN epoch: 21 loss: 0.571399
TRAIN epoch: 22 loss: 

KeyboardInterrupt: 